In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML 


In [2]:
df = pd.read_csv('./data/loan.csv', low_memory=False)

In [3]:
list(df)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

In [35]:
df['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'], dtype=object)

# Premise

We approach EDA from a perspective of an investor, whose interests are maximizing risk-adjusted returns. Here we will look at interest rate vs default rate, and then calculate a net adjusted annual return separated by the loan grade.

Next, we answer the question, are 36 month loans 


Lastly, we will build a model that can create a basket of investments that can outperform the market basket. These are the steps done and assumptions made.

    1. We first calculate the sharpe ratio using net adjusted return on all historical basket of loans. 
    2. We will build a regression model, using features chosen carefully through nested model testing, that will forecast the returns of all the loans in a validation dataset
    3. We will then use linear programming to set constraint and build a portfolio that maximizes the sharpe ratio with 10% of market liquidity
    4. We will then compare our model vs actual result to get a sense of whether we were able to outperform or not 

Additional analysis that can be done is accounting for changes in the risk free rate, and rebalancing of the portfolio year over year.

# Assumptions & Discussions

1. Funded Amount vs Funded Amount Inv: Where there are differences, lending tree stepped in to make up for the differences between loan amount and funed amount invested by investors. We will go ahead and use funded amount as a basis for all our analysis.
2. We assume all cash is deployed on year of x(has the most data) to reduce cash drag. We calculate IRR-> assuming we can reinvest at this rate 

# Exploratory Analysis

In [ ]:
https://www.lendingclub.com/public/images/0180344/content/aboutNAR/NAR.png

In [129]:
Image(url= "https://www.lendingclub.com/public/images/0180344/content/aboutNAR/NAR.png")

As an investor, you want to get a good grasp of two things of any investments, risk and reward. This analyis will mainly focus on that. 

Dissecting the data to too many categorical values can 

In [21]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,loan_status_flag
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


First we calculat the default percentage, we will include the "does not meet..." rows as valid loans, and throw out rest of the data that we don't have complete information on.

In [33]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [5]:
df['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'], dtype=object)

In [106]:
df['issue_yr']=pd.to_numeric(df['issue_d'].str.extract("([0-9]+)",expand=False))

In [113]:
searchfor = ['Fully Paid', 'Charged Off', 'Default']
df_filtered = df[df['loan_status'].str.contains('|'.join(searchfor))].query("term == ' 36 months' and issue_yr <=2012")
df_filtered['loan_status'].unique()

array(['Fully Paid', 'Charged Off',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Default'], dtype=object)

In [114]:
df_filtered['loan_status_flag'] = np.where(df_filtered['loan_status'].str.contains('Fully Paid') ,1,0)

In [115]:
df_filtered['loan_status_flag'].mean()

0.870028579823179

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,loan_status_flag
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,1072053,1288686,3000.0,3000.0,3000.0,36 months,18.64,109.43,E,E1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [116]:
df_filtered['issue_yr']=df_filtered['issue_d'].str.extract("([0-9]+)",expand=False) 

In [123]:
df_group = df_filtered.groupby(['grade'])['loan_status_flag'].mean()
df_group

grade
A    0.934345
B    0.880474
C    0.832614
D    0.796342
E    0.776184
F    0.709302
G    0.659836
Name: loan_status_flag, dtype: float64

(-1, 8)